## Question 1

### a.
<br> $ F = \{V, \sum, R, S\} $
<br> V = \{'S', 'P', 'V', 'A', 'N'\} 
<br> $ \sum = $ \{'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 
           'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 
           'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 
           'T', 'U', 'V', 'W', 'X', 'Y', 'Z' \} 
<br> R = rules = \{
        'S' -> 'PVAN' <br>
        'N' -> 'apple' | 'banana' | 'orange' | 'grapefruit' | 'coconut' | 'peach' | 'pear' | 'lychee' | 'rasberry' | 'watermelon' |<br>
        'V' -> 'eat' | 'buy' | 'love' | 'cut' |'digest' | <br>
        'A' -> 'big' | 'small' | 'sweet' | 'sour' | 'delicious' | <br>
        'P' -> 'I' | 'he' | 'she' | 'they' | 'we' | 'you' | 'who' |<br>
        \} <br>
S = \{'S'\}

### b.

The grammar above, F, describing the language of some simple english sentences revolving around fruit is unambiguous. Each string that can be generated only has one derivation, meaning we can't take two different "routes" to reach the same string. We know this because the starting rule, "S->PVAN" only derives P,V,A, and N in that exact order. Furthermore, P,V,A, and N only generate sets of terminals (words) that are unique in the rule structure. Thus, the grammar is unambiguous

### c.

Please refer to code cell below for formal definition of an equivalent NPDA. I will provide an informal definition here. We follow Sisper's method and proof that every CFG has an equivalent PDA. The code takes in a dictionary as a set of the rules for our CFG. It will iterate through, creating the transitions required for our PDA. The most important state is "q0" which is essentially a looping state. For each individual rule, there will be a loop here that replaces each variable, pushing the right-hand side of the rule onto the stack instead. There will also be another kind of loop that upon an input (terminal) matching the top of the stack, this character will be popped off the stack. Thus, the larger picture is each variable will continually be replaced by its right-hand-side rule until only terminals remain, then non-deterministically check if an input value also matches top of stack. If a string were to be generated by the grammar, the PDA will eventually become an empty stack, leading to accept state, accepting the string.

In [72]:
# rules are strategically converted to a python dictionary with values being list
# the properties of CFG rules closely resembles the immutability of this data structure
# or at least is sufficient for our purposes
rules_F = {
    'S': ['P V A N'],
    'N': ['apple', 'banana', 'orange', 'grapefruit', 'coconut', 'peach', 'pear', 'lychee', 'raspberry', 'watermelon'],
    'V': ['eat', 'buy', 'love', 'cut', 'digest'],
    'A': ['big', 'small', 'sweet', 'sour', 'delicious'],
    'P': ['I', 'he', 'she', 'they', 'we', 'you', 'who']
}

In [73]:
# produce input characters and stack symbols
def characters(rules):
    variables = []
    terminals = []
    for var in rules:
        variables.append(var)
    for i in rules:
        for j in rules[i]:
            for k in j:
                if k not in variables:
                    terminals.append(k)
    # unique values only
    variables = list(set(variables))
    terminals = list(set(terminals))
    return variables, terminals

# parses through a CFG dictionary, producing transitions for NPDA using Sisper's Lemma 2.21
def cfg_to_npda_helper(grammar):
    
    # initialize transitions 
    transitions = {
        'q_start': {
            '': {
                '$': {('q0', ('S', '$'))},
            },
            # this transition is for testing only. Input "X" should easily be accepted
            'X': {'$':{('q_start', ('Z', '$'))}}
        },
        'q0': {
            '': {
                '$':{('q_accept', '')}
            },
        },
    }
    # get variables and terminals from characters function defined above
    variables, terminals = characters(grammar)
    
    # I won't explain all the code, but the idea is to loop through every rule in the dictionary
    # A few patterns are observerd, for example for rules of length 1 etc. These are implemented
    # for more efficient code. Otherwise, refer to Sisper Lemma 2.21 for details on what is happening
    for variable in variables:
        transitions['q0'][''][variable] = set()
    for terminal in terminals:
        transitions['q0'][terminal] = {terminal: {('q0', '')}}
    counter = 0
    state = 'q'+str(counter)
    for var in grammar:
        for rule in grammar[var]:
            if len(rule) == 1:
                # for rules such as O -> E or O -> b etc.
                transitions['q0'][''][var].add(('q0', rule))
            elif rule == '':
                # for rules such as O -> '', empty string transitions
                transitions['q0'][''][var].add(('q0', ''))
            else:
                counter += 1
                state = 'q'+str(counter)
                transitions['q0'][''][var].add((state, rule[-1]))
                temp = -1
                
                # we want the add the characters of the rule in reverse, creating new state each time
                # except for the first character, that will transition back to q0
                for character in rule[::-1][:-2]:
                    temp -= 1
                    transitions[state] = {'':{character:{('q'+str(counter+1), (rule[temp], character))}}}
                    counter += 1
                    state = 'q'+str(counter)
                transitions[state] = {'':{rule[1]:{('q0', (rule[0], rule[1]))}}}
    # instead of re-writing it by hand later, use elegant power of code to get all states
    states_set = set()
    for i in transitions:
        states_set.add(i)
    states_set.add('q_accept')
    return transitions, states_set

In [74]:
variables_F, terminals_F = characters(rules_F)
npda_F_transitions, states_F_set = cfg_to_npda_helper(rules_F)

# Formal Definition of equivalent machine to cfg_F:
#print('variables: ', variables_F)
#print('terminals: ', terminals_F)
#print('transitions: ', npda_F_transitions)
#print('states: ', states_F_set)

In [75]:
from automata.pda.npda import NPDA

def create_cfg_to_npda(variables, terminals, transitions, states):
    npda = NPDA(
        states=states,
        # append a few extra characters, for testing, and initial stack symbol
        input_symbols=set(terminals+['X']),
        stack_symbols=set(variables+terminals+['$','X']),
        transitions=transitions,
        initial_state='q_start',
        initial_stack_symbol='$', # initialize stack with "$"
        final_states={'q_accept'},
        # if recognizable string, NPDA should end both on the accept state, as well as with empty stack
        acceptance_mode='both'
    )
    return npda

In [76]:
# Question 1.d)
npda_F = create_cfg_to_npda(variables_F, terminals_F, npda_F_transitions, states_F_set)

In [77]:
# comprehensive tests for npda_F, noting when npda should and should not accept
# would be nice one day to make this more grammatically sound. "He eat" kills me inside
assert(npda_F.accepts_input('I eat sweet apple') == True)
assert(npda_F.accepts_input('you cut small raspberry') == True)
assert(npda_F.accepts_input('he love delicious peach') == True)
assert(npda_F.accepts_input('I cut big cabbage') == False)
assert(npda_F.accepts_input('Chretien eat sweet apple') == False)
assert(npda_F.accepts_input('I eat sweet Apple') == False)

## Question 2

### a.

We can build our translation automaton through a Mealy machine with no accept state. A Mealy Machine functions closely to a Finite State Automaton. However, upon input, the mealy machine will instead print or return an output. Our autmaton will work as so. q0 will transition with all inputs from P. The translation is printed if the transition is made. The next layer, all receiving nodes from inputs in P, will transition with all inputs from V; again, if an input matches a possible transition, make the transition and print translation. We repeat this for a third and fourth layer of nodes for all words in A and N. In all, we will have (P$\times$V$\times$A$\times$N)+1 total number of nodes.

Output translations as so: 

\{
'apple' -> 'apfel'
'orange' -> 'orange'
'grapefruit' -> 'grapefruit'
'coconut' -> 'kokosnuss'
'peach' -> 'pfirsich'
'pear' -> 'birne'
'lychee' -> 'litschi'
'rasberry' -> 'himbeere'
'watermelon' -> 'wassermelone'
'eat' -> 'essen'
'buy' -> 'kaufen'
'love' -> 'liebe'
'cut' -> 'schneiden'
'digest' -> 'verdauen'
'big' -> 'gro&szling;'
'small' -> 'klein'
'sweet' -> 's&uuml;ss'
'sour' -> 'sauer'
'delicious' -> 'lecker'
'I' -> 'ich'
'he' -> 'er'
'she' -> 'sie'
'they' -> 'sie'
'we' -> 'wir'
'you' -> 'sie'
'who' -> 'wer'
\}
$M = \{S, S_0, \Sigma, \Lambda, T, G \} $

$S = \{q_0, q_1, q_2, ... , q_n |$ where $n= |P\times V\times A\times N|+1 $ \}

$S_0 = \{q_0\}$

$\Sigma$ = \{'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'\}

$\Lambda$ = \{'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '&auml;', '&ouml;', '&uuml;', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',  '&Auml;', '&Ouml;', '&Uuml;', '&szlig;'\}

$T = \{S\times \sigma \rightarrow S |$ as described above \}

$G = \{S\times \lambda \rightarrow S |$ direct translations as shown above \}

### b.

The main computational drawback to the approach above is its how many states it will. If we were to build a sentence, of say 20 words, with 8 parts of speech, 100 words in each POS, our automata will have 100^20 states! And when we factor in realistically how grammar works, for example following a verb can be both proper nouns or objective pronouns, then this number becomes even more massive! This is obviously highly inefficient, we're going word by word so we never know what is going to come next, and have to make sure that following each word, there is a transition to all the possible words that can come after it. Furthermore, there is no context of a word being taken in account. If we have a word, talk, it can either be "I talk" or "He talks." These grammatical cues and differences, tenses etc. are unaccounted for, and arguably if you translate word-by-word, impossible to account for.

## Question 3

### a. 
<br> $ G = \{V, \sum, R, S\} $
<br> V = \{'S', 'I', 'E', 'O', 'F', 'V', 'D', 'M', 'C', 'U', 'N'\} 
<br> $ \sum = $ \{'f', '(', 'x', ')', '=', 'x', ',', 'y', ')', '=', '(', ')', '(', ')', '+', '-', '*', '/', '^', 's', 'i', 'n', '(', ')', 'c', 'o', 's', '(', ')', 't', 'a', 'n', '(', ')', 's', 'q', 'r', 't', '(', ')', 'l', 'o', 'g', '(', ')', 'x', '(', ')', '(', ')', '+', '-', '*', '/', 's', 'i', 'n', '(', ')', 'c', 'o', 's', '(', ')', 't', 'a', 'n', '(', ')', 's', 'q', 'r', 't', '(', ')', 'l', 'o', 'g', '(', ')', 'x', 'y', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0'\} 
<br> R = rules = \{
        'S' -> 'f(I' <br>
        'I' -> 'x)=E' | 'x,y)=D' |<br>
        'E' -> 'EO' | '(EO)' | '(E)O' | 'F' |'V' | <br>
        'O' -> '+E' | '-E' | '\*E' | '/E' | '^E' | $ \epsilon\ $ <br>
        'F' -> 'sin(E)' | 'cos(E)' | 'tan(E)' | 'sqrt(E)' | 'log(E)'<br>
        'V' -> 'NV' | 'N' | 'x' |<br>
        'D' -> 'DM' | '(DM)' | '(D)M' | 'C' |'U' <br>
        'M' -> '+D' | '-D' | '\*D' | '/D' | $ \epsilon\ $ <br>
        'C' -> 'sin(D)' | 'cos(D)' | 'tan(D)' | 'sqrt(D)' | 'log(D)' <br>
        'U' -> 'NU' | 'N' | 'x' | 'y' |<br>
        'N' -> '1' |'2' |'3' |'4' |'5' |'6' |'7' |'8' |'9' |'0'<br>
        \} <br>
S = \{'S'\}

### b.
For the formal description of the equivalent machine to the context-free grammar above, which is a non-deterministic pushdown automata, please refer to the NPDA below, "npda_function_generator." I will provide a brief overview here for an informal description. The context-free language recognized by the grammar above is every string of the sort "f(x)=E" or "f(x,y)=D," with E and D being any expression generatable by natural numbers, basic operators (+,-,$\times$, $\div$, $\exp$) and functions: $\sin$, $\cos$, $\tan$, $ \sqrt{} $, and $\log$ . The difference being that single input x is valid for E and two inputs, x and y, are valid for D. We start by converting the grammar above, G, to the proper corresponding NPDA according to Sisper Lemma 2.21. We already know that every CFG has an equivalent PDA, so this should be possible. Once we get the transitions, we can simply utilize automata.lib to implement the final steps of our getting our NPDA simulator.


### c.

In [78]:
# Accepted functions: sin, cos, tan, sqrt, log
# Accepted operators: +, -, /, *, ^
rules_G = {'S': ['f(I'],
        'I': ['x)=E', 'x,y)=D'],
        'E': ['EO', '(EO)', '(E)O', 'F','V'],
        'O': ['+E', '-E', '*E', '/E', '^E', ''],
        'F': ['sin(E)', 'cos(E)', 'tan(E)', 'sqrt(E)', 'log(E)'],
        'V': ['NV', 'N', 'x'],
        'D': ['DM', '(DM)', '(D)M', 'C','U'],
        'M': ['+D', '-D', '*D', '/D', ''],
        'C': ['sin(D)', 'cos(D)', 'tan(D)', 'sqrt(D)', 'log(D)'],
        'U': ['NU', 'N', 'x', 'y'],
        'N': ['1','2','3','4','5','6','7','8','9','0']
        }

# I should say chomsky-esque. I tried my best but it gets so large, can't process in npda
rules_G_chomsky = {
        'S': ['f(I'],
        'I': ['x)=E', 'x,y)=D'],
        'E': ['xO', '(xO)', '(x)O', '(x)', 'sin(x)', 'cos(x)', 'tan(x)', 'sqrt(x)', 'log(x)', 'EO', '(EO)', '(E)O', '(E)', 'sin(E)', 'cos(E)', 'tan(E)', 'sqrt(E)', 'log(E)', 'EV'],
        'V': ['1','2','3','4','5','6','7','8','9','0'],
        'O': ['BE'],
        'B': ['+', '-', '*', '/', '^'],
        'D': ['YM', '(YM)', '(Y)M', '(Y)', 'sin(Y)', 'cos(Y)', 'tan(Y)', 'sqrt(Y)', 'log(Y)', 'DM', '(DM)', '(D)M', '(D)', 'sin(D)', 'cos(D)', 'tan(D)', 'sqrt(D)', 'log(D)', 'DV'],
        'M': ['CD'],
        'C': ['+', '-', '*', '/', '^'],
        'Y': ['x', 'y']
        }

In [79]:
variables_G, terminals_G = characters(rules_G)
npda_G_transitions, states_G_set = cfg_to_npda_helper(rules_G)

# Formal Definition of equivalent machine to cfg_G:
#print('variables: ', variables_G)
#print('terminals: ', terminals_G)
#print('transitions: ', npda_G_transitions)
#print('states: ', states_G_set)

In [80]:
npda_G = create_cfg_to_npda(variables_G, terminals_G, npda_G_transitions, states_G_set)

In [81]:
assert(npda_G.validate() == True)
assert(npda_G.accepts_input('f(x)=8+8+10*x'))
assert(npda_G.accepts_input('f(x,y)=(sin(x)+tan(x/y))*x') == True)
assert(npda_G.accepts_input('f(x)=x+x+x') == True)
assert(npda_G.accepts_input('f(x,y,z)=1+2+3') == False)
# the more nested functions there are the longer the runtime. Should be right though
#print(list(npda_function_generator.read_input_stepwise('f(x)=sin(tan(log(cos(5*x^100))))')))
#print(npda_function_generator.read_input('f(x,y)=2+2'))

### d.

We will use python's eval() function to help parse our string and generate appropriate function. No need to reinvent the wheel here.

In [82]:
from math import sin, cos, tan, log10, sqrt, pi

# function that outputs function generated from our input string, if recognized by NPDA
def output_function(npda, input_string):
    if npda.accepts_input(input_string):
        
        #parse our string, if accepted, return function that evaluates left hand side
        parse = input_string.find('=')
        parse_string = input_string[parse+1:]
        
        # '^' is in our language but python evaluates '**' instead
        parse_string = parse_string.replace('^', '**').replace('log', 'log10')
        
        # default values so x and y are optional. Our npda will reject if x,y appear in f(x) function
        def evaluate(x=1, y=1):
            x = x
            y = y
            z = eval(parse_string)
            return z
        return evaluate
    else:
        return 'Input string invalid'
# tests
function1 = output_function(npda_G, 'f(x,y)=(sin(x)+cos(y))*5+10')   
function2 = output_function(npda_G, 'f(x)=10*x+log(10^x)')     

In [83]:
# ensure function generator works
assert(function1(x=0, y=0) == 15.0)
assert(function2(x=5) == 55.0)
assert((function2(x=2) == 30) == False)
assert((function2(x=2) == 22) == True)

In [84]:
# A test NPDA to help simplify and debug syntaxical errors. Otherwise ignore
from automata.pda.npda import NPDA
test_g = {'S':['aTb', 'b'],
          'T':['Ta', '']
    }
npda_transitions_test, states_set_test = cfg_to_npda(test_g)
print(npda_transitions_test)
states_set_test.add('q_accept')
variables_test, terminals_test = characters(test_g)
test_g_npda = NPDA(
    states=states_set_test,
    input_symbols=set(variables_test+terminals_test+['X', 'Z']),
    stack_symbols=set(variables_test+terminals_test+['X', '$', 'Z']),
    transitions={
        'q_start': {
            '': {'$': {('q0', ('S', '$'))}}, 
            'X': {'$': {('q0', ('b', '$'))}}
        }, 
        'q0': {
            '': {
                '$': {('q_accept', ('', '$'))}, 
                 'S': {('q1', 'b'), ('q0', 'b'), ('q_accept', '')}, 
                 'T': {('q0', ''), ('q3', 'a')},
                'Z':{('q_accept', '$')}
            }, 
            'a': {
                'a': {('q0', '')},
                'b': {('q_accept', '')}
                
            }, 
            'b': {
                'b': {('q0', '')}
            }
        }, 
        'q1': {
            '': {
                'b': {('q2', ('T', 'b'))}
            }
        }, 
        'q2': {
            '': {
                'T': {('q0', ('a', 'T'))}
            }
        }, 
        'q3': {
            '': {'a': {('q0', ('T', 'a'))}}
        }
    },
    initial_state='q_start',
    initial_stack_symbol='$',
    final_states={'q_accept'},
    acceptance_mode='final_state'
)
test_g_npda.read_input('Xa') 

{'q_start': {'': {'$': {('q0', ('S', '$'))}}, 'X': {'$': {('q_start', ('Z', '$'))}}}, 'q0': {'': {'$': {('q_accept', '')}, 'T': {('q0', ''), ('q3', 'a')}, 'S': {('q0', 'b'), ('q1', 'b')}}, 'a': {'a': {('q0', '')}}, 'b': {'b': {('q0', '')}}}, 'q1': {'': {'b': {('q2', ('T', 'b'))}}}, 'q2': {'': {'T': {('q0', ('a', 'T'))}}}, 'q3': {'': {'a': {('q0', ('T', 'a'))}}}}


{PDAConfiguration('q0', 'Xa', PDAStack('$', 'b')),
 PDAConfiguration('q1', 'Xa', PDAStack('$', 'b')),
 PDAConfiguration('q_accept', '', PDAStack('$',)),
 PDAConfiguration('q_accept', 'Xa', PDAStack('$',))}

## Question 4

I will be looking at the (former) statsitical-translation machine Google Translate. SMT generates translations based on statistical models, whose parameters are derived from bilingual text corpora. Thus, SMT is a fully supervised process. The corpora (just large bodies of text) are all preselected, manipulated, and prepared "by hand," (likely aided by some well-defined functions), usually extracted from large international organizations such as the EU, UN, World Bank etc. that often have build massive multilingual documents. Since these were originally human translated and edited for legal/political purposes, they serve as a pretty accurate source of pre-classified data. As for how the model is actually constructed, phrases, of length to five (usually done through a process of ngrams, so if ngrams=3, every series of three words), are assessed based on their frequency of occuring in a document. The statistical model says more or something something along the lines of "which foreign sentence, 'f', produces its english translation "e." In other words, what is the probability of e given f, P(e|f). The statistical model allows us to infer if there's any relationship among phrases words in between the two languages

Rule-based translations relies on many many built-in linguistic and grammatical rules, and even more bilingual dictionaries. This in effect tells us how a word or phrase in the source language should be read in another language. RBMT is based on morphological, syntactic, and semantic factors, working better, or even requiring, a near-full set of dictionary translations. Can only do what it is told to do. I would argue SMT ought to be preferred over RBMT. They're built on phrase-based systems, offering more fluid and "better" translations. This is also in part due to the fact that they're trained on human work, how humans actually use their respective languages. Lastly, statistical modeling is far more flexible than rule-based approaches, they adopt a bottom-up approach whereas rule-based is very top down. There's no need for a team of linguistic specialists to seminate their expertise on sentence structures, but instead, we generate predictions that two phrases are indeed related based on a probalistically acceptable threshold. I'd imagine that rule-based machines would require a context-free language generator/recognizer. They're not able to recognize anything outside of the basic unit of words; so the rules they recognize are likely context free. SMT on the other hand probably requires context-sensitive languages. If I see the word "water," I can't just translate the english noun to its french equivalent "eau," but instead, based on its context, it could follow a subjective noun, making it a verb. If this were the case, the more appropriate translation would be "arroser," meaning "to water." Furhtermore, if the subjective pronoun was "I water," the correct translation and conjugation should instead be "arrose."


## Question 5: HC tags

#probability - Question 4 asked us to dive into the world of statistical machine tranlsations. Although not given the oppurtunity to create any probability models ourselves, in this question I apply knowledge and understanding of SMT's, which themselves are applications of conditional probability. The summary is clear, concise, and ultimately gets at the heart of what is going on with these technologies and algorithms. Furthermore, the benefits to using statistical models in ML/NLP are highlighted, and the larger picture view of how probability can be used for prediction/classification ML problems is provided.

#heuristics - similar to the previous assignment, I tried really hard to apply this HC to make my own life easier. A simple heuristic was employed so as to save time with the general task. The intuition was to abstract away repetitive tasks with other code functions. While last time I used this heuristic of "abstract when possible" was to save time typing, this time I realized I could use it for a practical implementation of Sisper's CFG to PDA method. Similarly, I recognized the repetitive nature of the algorithm that justifies the use of this heuristic, since code is optimized for looping tasks. There was also another pratical consideration. I tried to draw the PDA by hand, and upon each mistake, I more or less had to restart, rename all the states etc. so as to make the final work not to messy. By abstracting away the need for manual re-edits, code made sense as well. I wasn't declaring all the states individually, but instead iteratively, meaning the states and transitions would be generated in a systematic way. In the code, I also abstract away needing to type all variables/terminals.  Most importantly, I abstracted away the need to type the hundreds of transitions myself. This was the most important. All of the transitions in Sisper's PDA follow general patterns, and they exist based on their structure in the rules. Thus, code could be used instead to suplement the process. The success of the algorithm is a testament to the application of heuristics  

#levelsofanalysis - I'm a bit surprised myself that I used this HC, but it was all thanks to one bug, for which I stayed up to sunrise (literally) trying to solve. Looking back, the bug was such a minute typo its laughable, but either way, without levels of analysis, I would have never been able to resolve it. Here is the bug:

```
def cfg_to_npda_helper(grammar):
    transitions = {
        'q_start': {
            '': {
                '$': {('q_0', ('S', '$'))},
            },
            'X': {'$':{('q_start', ('Z', '$'))}}
        },
        'q0': {
            '': {
                '$':{('q_accept', '$')}
            },
        },
    }
```   

No matter what I did, I could not find out why my NPDA was not accepting what it should be accepting. The first level where the problem might arise is conceptually: perhaps Sisper got it wrong. But then I thought, "the guy's pretty smart, he probably knows what he's talking about, and this is his book's 3rd edition if something wrong still went unchecked that would be surprising," and wanted to dismiss the possibility, but nonetheless I verified that transitions outputted should have accepted my test string, and it was fine. If the issue wasn't with Sisper's methodology in generating the transitions, then perhaps the error lies at the dependency level. If I used automata-lib's syntax wrong, or the data structure of transitions were off, then perhaps what the transitions weren't transitioning how they're supposed to. This too I double-checked, and even built a test NPDA and everything worked there. This suggested my analysis had to go to deeper layer still. I peeled back each loop of the function to make sure it looked ok, and it did. This then meant that the only remaining level of my code to check is the hard-code. This refers to the initial variables I set in the beginning. These levels although usually easily noticeable when flawed, are particularly dangerous because of how they impact/interact with the other levels of the function. They often get called upon, define higher levels, and initializes the functionality of the other levels. That last part I realized I did not check: perhaps my initialization was wrong which created a ripple effect through the other layers of the code, causing it all to not even really run in the first place. Surely enough, as you can see above, I label 'q_0' instead of 'q0.' Meaning even with all the other levels being accurate in principle, they were running on a state that never existed. It was a rather costly typo, several hours of work in fact, but by finallt resorting to levels of analysis in a novel way, with my function itself, and understanding how the different levels interact with one another, the diagnosed problem was finally resolved.